In [13]:
import tensorflow
from keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten, Bidirectional, Concatenate
from keras.models import Model
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [81]:
batch_size = 16
max_sequence_length = 384
max_word_length = 20
embedding_size = 128
lstm_units = 256
num_classes = 14

directory_train='/content/sample_data/training-data'
directory_test='/content/sample_data/testing-data'
directory_all='/content/sample_data/all-data'
model_path='bi_best_model.keras' #modelin konumu
model_test_path='/content/sample_data/deneme-verileri'#modelin testi için kullanılacak verinin konumu
model_test_path_labeled='/content/sample_data/denem-verileri-etiketli' # model testinin accuracy hesaplanması için kullanılacak veriler

In [5]:
train_test_texts=[]

for filename in os.listdir(directory_all):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_all, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            content = re.sub(r'<start_".*?">', '', content)
            content = re.sub(r'<end_".*?">', '', content)
            txt_data = content.split()
            train_test_texts.extend(txt_data)

tokenizer = Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(train_test_texts)

vocab_len = len(tokenizer.word_index) + 1

In [6]:
#SAVE TOKENIZER
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(tokenizer.word_index)

{'0': 1, 'A': 2, '1': 3, 'E': 4, 'T': 5, 'I': 6, '2': 7, 'R': 8, 'N': 9, 'L': 10, 'K': 11, 'S': 12, '*': 13, 'O': 14, '5': 15, 'M': 16, ':': 17, '.': 18, '3': 19, '8': 20, '4': 21, '9': 22, 'D': 23, '6': 24, 'İ': 25, 'U': 26, '7': 27, ',': 28, 'B': 29, 'Y': 30, 'C': 31, 'P': 32, 'V': 33, 'e': 34, 'i': 35, 'a': 36, 'H': 37, 'G': 38, 'Z': 39, 'F': 40, 'r': 41, '/': 42, 'Ş': 43, 'n': 44, 'o': 45, 'l': 46, '%': 47, 't': 48, 'Ü': 49, 's': 50, '-': 51, 'k': 52, 'd': 53, 'm': 54, 'X': 55, 'u': 56, 'y': 57, 'c': 58, 'z': 59, 'ı': 60, 'h': 61, 'g': 62, 'b': 63, 'Ç': 64, 'w': 65, '#': 66, 'Ğ': 67, 'Ö': 68, 'ş': 69, 'v': 70, 'ü': 71, 'p': 72, 'W': 73, ')': 74, '(': 75, 'J': 76, 'f': 77, 'x': 78, 'ğ': 79, '$': 80, '»': 81, "'": 82, '+': 83, 'ç': 84, 'Q': 85, '!': 86, 'Í': 87, 'ж': 88, 'ö': 89, '=': 90, '&': 91, 'j': 92, '\\': 93, 'Т': 94, 'О': 95, 'Р': 96, '"': 97, '>': 98, 'К': 99, 'Ș': 100, 'Ø': 101, 'q': 102, 'Ú': 103, '@': 104, '|': 105, 'Á': 106, ';': 107, '×': 108, '<': 109, '[': 110, '_': 1

In [7]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
                txt_data = content.split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)

    #tokenizer = Tokenizer(char_level=True, lower=False)
    #tokenizer.fit_on_texts(all_texts_extend)

    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))


    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'

    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)

    return all_padded_sequences, tokenizer.word_index


#   TRAIN
directory_path = directory_train
padded_sequences, word_index = process_text_files(directory_path)
padded_arrays = []
for arr in padded_sequences:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays.append(padded_arr)

#  TEST
directory_path_test = directory_test
padded_sequences_test, word_index_test = process_text_files(directory_path_test)
padded_arrays_test = []
for arr in padded_sequences_test:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays_test.append(padded_arr)


np.set_printoptions(threshold=np.inf)

#son array'i numpy array'ine çevir
x_train=np.array(padded_arrays)
x_test=np.array(padded_arrays_test)

print(x_train.shape)
print(x_test.shape)

np.set_printoptions(threshold=1000)

(547, 384, 20)
(547, 384, 20)


In [8]:
global wordsList,wordsIndextList

wordsList = []
wordsIndextList = []
max_sequence_length = 384


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]



def one_hot_encode_index_list():
    oneHotEncodedList = []
    unique_labels_len = len(sorted(set(label_list2)))
    for i in wordsIndextList:
        bitlist = list(0 for i in range(unique_labels_len))
        bitlist[i] = 1
        oneHotEncodedList.append(bitlist)

    for i in  range(max_sequence_length-len(oneHotEncodedList)):
        oneHotEncodedList.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0])
    return oneHotEncodedList


def splitfunction(text:str):
    global wordsList,wordsIndextList
    wordsList = []
    wordsIndextList = []
    words = text.split()
    for word in words:
        wordsList.append(word)
        wordsIndextList.append(1)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag
    return None



def tagingWords():
    control = None
    counter = 0
    for word in wordsList:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndextList[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1


def oneHotEncodedFunction(texts):
    splitfunction(texts)
    tagingWords()
    return np.array(one_hot_encode_index_list())



# TRAIN
folder_path_train =  directory_train
file_list_train = os.listdir(folder_path_train)
y_train= []
for file_name in file_list_train:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_train, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)

            if len(y_train) == 0:
                y_train = np.array([result])
            else:
                y_train = np.concatenate((y_train, [result]), axis=0)


# TEST
folder_path_test =  directory_test
file_list_test = os.listdir(folder_path_test)
y_test= []
for file_name in file_list_test:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_test, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)

            if len(y_test) == 0:
                y_test = np.array([result])
            else:
                y_test = np.concatenate((y_test, [result]), axis=0)


print(y_train.shape)
print(y_test.shape)

(547, 384, 14)
(547, 384, 14)


In [9]:
# LSTM Modeli

input_shape = (max_sequence_length, max_word_length)
input_layer = Input(shape=(max_sequence_length,max_word_length))
print(input_layer.shape)


(None, 384, 20)


In [10]:
char_embedding = Embedding(input_dim=vocab_len, output_dim=embedding_size)(input_layer)
print(char_embedding.shape)

(None, 384, 20, 128)


In [11]:
reshaped_embedding = tensorflow.reshape(char_embedding, (-1, max_word_length, embedding_size))
print(reshaped_embedding.shape)

(None, 20, 128)


In [12]:
lstm_units = 64
char_lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(units=lstm_units, return_sequences=True, return_state=True))(reshaped_embedding)
print(char_lstm)

char_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(units=lstm_units, return_sequences=True, return_state=True))(
    char_lstm, initial_state=[forward_h, forward_c, backward_h, backward_c])

state_h = Concatenate()([forward_h2, backward_h2])
state_c = Concatenate()([forward_c2, backward_c2])

print(char_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(None, 20, 128), dtype=tf.float32, name=None), name='bidirectional/concat:0', description="created by layer 'bidirectional'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 128), dtype=tf.float32, name=None), name='bidirectional_1/concat:0', description="created by layer 'bidirectional_1'")


In [ ]:
reshaped = tensorflow.reshape(state_h, (-1, max_sequence_length, 2 * lstm_units))
print(reshaped.shape)

(None, 384, 128)


In [ ]:
word_lstm1, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(units=lstm_units, return_sequences=True, return_state=True))(reshaped)
print(word_lstm1)

word_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(units=lstm_units, return_sequences=True, return_state=True))(word_lstm1, initial_state=[forward_h, forward_c, backward_h, backward_c])
print(word_lstm2)

state_h = Concatenate()([forward_h2, backward_h2])
state_c = Concatenate()([forward_c2, backward_c2])

KerasTensor(type_spec=TensorSpec(shape=(None, 384, 128), dtype=tf.float32, name=None), name='bidirectional_2/concat:0', description="created by layer 'bidirectional_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 384, 128), dtype=tf.float32, name=None), name='bidirectional_3/concat:0', description="created by layer 'bidirectional_3'")


In [ ]:
classification_output = Dense(units=num_classes, activation='softmax')(word_lstm2)
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(None, 384, 128), dtype=tf.float32, name=None), name='bidirectional_3/concat:0', description="created by layer 'bidirectional_3'")


In [ ]:
model = Model(inputs=input_layer, outputs=classification_output)

In [ ]:
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras as keras

checkpoint = ModelCheckpoint('bi_best_model.keras', monitor='val_accuracy', save_best_only=True)

opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 20)]    0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 384, 20, 128) 25088       input_1[0][0]                    
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None, 20, 128)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 20, 128), (N 98816       tf.reshape[0][0]                 
______________________________________________________________________________________________

In [ ]:

model.fit(x_train, y_train, epochs=1000, batch_size=16, validation_data=(x_test, y_test), callbacks=[ checkpoint])

Epoch 1/1000
35/35 [==============================] - 18s 234ms/step - loss: 2.2554 - accuracy: 0.6677 - val_loss: 1.2587 - val_accuracy: 0.7306
Epoch 2/1000
35/35 [==============================] - 6s 172ms/step - loss: 0.8725 - accuracy: 0.7167 - val_loss: 0.6981 - val_accuracy: 0.7306
Epoch 3/1000
35/35 [==============================] - 6s 174ms/step - loss: 0.6211 - accuracy: 0.7470 - val_loss: 0.4143 - val_accuracy: 0.9249
Epoch 4/1000
35/35 [==============================] - 6s 172ms/step - loss: 0.2731 - accuracy: 0.9544 - val_loss: 0.2145 - val_accuracy: 0.9629
Epoch 5/1000
35/35 [==============================] - 6s 176ms/step - loss: 0.2001 - accuracy: 0.9602 - val_loss: 0.1796 - val_accuracy: 0.9647
Epoch 6/1000
35/35 [==============================] - 6s 179ms/step - loss: 0.1768 - accuracy: 0.9650 - val_loss: 0.1662 - val_accuracy: 0.9668
Epoch 7/1000
35/35 [==============================] - 6s 172ms/step - loss: 0.1666 - accuracy: 0.9658 - val_loss: 0.1595 - val_accuracy

In [ ]:
model.save("bi_model.keras")

In [83]:
# Test verilerini manuel accuracy hesaplama için hazırlama

global words,wordsIndext

words = []
wordsIndext = []
max_sequence_length = 384


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]


def splitfunction(text:str):
    global words,wordsIndext
    words = []
    wordsIndext = []
    texts = text.split()
    for word in texts:
        words.append(word)
        wordsIndext.append(1)
    for i in range(max_sequence_length-len(wordsIndext)):
        wordsIndext.append(0)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag
    return None



def tagingWords():
    control = None
    counter = 0
    for word in words:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndext[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1


def indexListFunction(texts):
    splitfunction(texts)
    tagingWords()
    return wordsIndext


folder_path_test =  model_test_path_labeled
file_list_test = os.listdir(folder_path_test)
accuracy_testing_data = []

for file_name in file_list_test:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_test, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = indexListFunction(text)
            if len(accuracy_testing_data) == 0:
                accuracy_testing_data = np.array([result])
            else:
                accuracy_testing_data = np.concatenate((accuracy_testing_data, [result]), axis=0)

In [84]:
# manuel accuracy hesaplama

def accuracy_calculation(npArray1,npArray2):
  different_elements = np.sum(npArray1 != npArray2)

  total_elements = npArray1.size
  difference_ratio = 1-different_elements / total_elements
  result = round(difference_ratio, 4)
  return result

In [95]:
#MODELİ ÇALIŞTIR
def model_run(directory):
    all_texts = []

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                txt_data = content.split()
                all_texts.append(txt_data)

    #TOKENIZER YÜKLE
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer_model = pickle.load(handle)
    print(tokenizer_model.word_index)

    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer_model.texts_to_sequences(all_texts[i]))

    max_word_length,trunc_type,padding_type = 20,'post','post'

    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)

    padded_arrays = []
    for arr in all_padded_sequences:
        pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
        padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
        padded_arrays.append(padded_arr)

    xtrain=np.array(padded_arrays)
    model = tensorflow.keras.models.load_model(model_path)

    prediction=model.predict(xtrain)
    print(prediction.shape)
    #------------------------------------------------------------
    label_list = ["Pad", "Others", "B_Comp", "I_Comp", "B_Date", "I_Date", "B_Time", "I_Time", "B_Receipt", "I_Receipt", "B_Tax", "I_Tax", "B_Amount", "I_Amount"]

    #deneme yapılacak fişin konumu
    with open("/content/sample_data/deneme-verileri/0.txt", "r", encoding="utf-8") as file:
        words = file.read().split()

    predicted_labels = np.argmax(prediction, axis=-1)

    #klasördeki ilk fişi temsil eder
    fis_1=predicted_labels[0]
    print(fis_1)

    print("-------------------------------------------------------------------------------")
    fis_1_etiketli = accuracy_testing_data[0]
    print(fis_1_etiketli)
    print("accuracy : {}".format(accuracy_calculation(fis_1_etiketli,fis_1)))  # gerçek veri ile model çıktısının karşılaştırıp accuracy döndürür
    print("-------------------------------------------------------------------------------")

    truncated_values = fis_1[:len(words)]#uzun olan diziyi kısalt ve iki dizi boyutunu eşle
    my_map = dict(zip(words, truncated_values))#kelimelere ve karşılarına predictionları 0 1 2 formatında maple

    result_map = {}
    for key, value in my_map.items():#anlamlı gözükmesi için 0 1 2 yerine etiket isimleri olacak şekilde maple
        index = min(value, len(label_list) - 1)
        label = label_list[index]
        result_map[key] = label

    print(result_map)
    for key, value in result_map.items():
        print(f'{key}: {value}')


In [96]:

#model_test_path içinde test edilecek veriler txt halde bulunmalıdır
model_run(model_test_path)

{'0': 1, 'A': 2, '1': 3, 'E': 4, 'T': 5, 'I': 6, '2': 7, 'R': 8, 'N': 9, 'L': 10, 'K': 11, 'S': 12, '*': 13, 'O': 14, '5': 15, 'M': 16, ':': 17, '.': 18, '3': 19, '8': 20, '4': 21, '9': 22, 'D': 23, '6': 24, 'İ': 25, 'U': 26, '7': 27, ',': 28, 'B': 29, 'Y': 30, 'C': 31, 'P': 32, 'V': 33, 'e': 34, 'i': 35, 'a': 36, 'H': 37, 'G': 38, 'Z': 39, 'F': 40, 'r': 41, '/': 42, 'Ş': 43, 'n': 44, 'o': 45, 'l': 46, '%': 47, 't': 48, 'Ü': 49, 's': 50, '-': 51, 'k': 52, 'd': 53, 'm': 54, 'X': 55, 'u': 56, 'y': 57, 'c': 58, 'z': 59, 'ı': 60, 'h': 61, 'g': 62, 'b': 63, 'Ç': 64, 'w': 65, '#': 66, 'Ğ': 67, 'Ö': 68, 'ş': 69, 'v': 70, 'ü': 71, 'p': 72, 'W': 73, ')': 74, '(': 75, 'J': 76, 'f': 77, 'x': 78, 'ğ': 79, '$': 80, '»': 81, "'": 82, '+': 83, 'ç': 84, 'Q': 85, '!': 86, 'Í': 87, 'ж': 88, 'ö': 89, '=': 90, '&': 91, 'j': 92, '\\': 93, 'Т': 94, 'О': 95, 'Р': 96, '"': 97, '>': 98, 'К': 99, 'Ș': 100, 'Ø': 101, 'q': 102, 'Ú': 103, '@': 104, '|': 105, 'Á': 106, ';': 107, '×': 108, '<': 109, '[': 110, '_': 1